In [1]:
# 获得拟合曲线
#调整了h高度
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress
from scipy.optimize import least_squares
import os
import warnings
warnings.filterwarnings('ignore')

# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
from scipy.optimize import least_squares
from scipy.stats import linregress
import warnings

# --- 参数空间定义 ---
R_list = np.array([0.5, 1, 2, 5, 10])
GAP_list = np.array([-0.1, 0, 0.1])
W_list = np.array([0.3, 0.5, 0.7])

# --- 定义幂函数拟合残差 ---
def PowerFunc(params, X, y):
    """拟合函数：wse = wse0 + a * width^b"""
    wse0, a, b = params
    return y - (wse0 + a * X**b)

# --- soft_l1 损失函数及其导数 ---
def Loss(z):
    rho = np.zeros((3, len(z)))
    rho[0] = 2 * ((1 + z)**0.5 - 1)              # 函数值
    rho[1] = (1 + z)**(-0.5)                     # 一阶导数
    rho[2] = -0.5 * (1 + z)**(-1.5)              # 二阶导数
    rho[:, 0] *= (len(z) - 2) / weight * (1 - weight) / 2   # 首点调整
    rho[:, 1] *= (len(z) - 2) / weight * (1 - weight) / 2   # 尾点调整
    return rho

# --- 数据读取 ---
df_s3all = pd.read_csv('swot_s3.csv').drop_duplicates(subset=['STCD', 'time'])
df_attr = pd.read_csv('attr/riv_attr.csv').set_index('COMID')
match = pd.read_csv('pre-data/COMID-station-STCD.csv').drop_duplicates()
df_w = pd.read_csv('pre-data/w50.csv').merge(match, on='station', how='inner').set_index('STCD')

# --- 构建参数组合表 ---
df_res = df_s3all.drop_duplicates('STCD')[['STCD', 'COMID']]
df_res = pd.concat([df_res] * (len(R_list) * len(GAP_list) * len(W_list)), ignore_index=True)
df_res = df_res.sort_values('STCD')
stationids = df_res['STCD'].unique()
df_res['R'] = R_list[np.tile(np.repeat(np.arange(len(R_list)), len(GAP_list)*len(W_list)), len(stationids))]
df_res['GAP'] = GAP_list[np.tile(np.repeat(np.arange(len(GAP_list)), len(W_list)), len(R_list)*len(stationids))]
df_res['W'] = W_list[np.tile(np.arange(len(W_list)), len(GAP_list)*len(R_list)*len(stationids))]
df_res = df_res.set_index(['STCD', 'R', 'GAP', 'W'])

# --- 主循环：每个站点 ---
for s in stationids:
    print(f'\r... processing {np.where(stationids==s)[0][0]+1}/{len(stationids)} ...', end='')
    df_s3 = df_s3all[df_s3all['STCD'] == s].copy()
    st, comid = df_s3.loc[df_s3.index[0], ['STCD', 'COMID']]

    # 获取流量、坡度、宽度三元组
    try:
        q50 = df_attr.loc[comid, 'q50']
        slp = df_attr.loc[comid, 'Slope']
        w50, w_low, w_high = df_w.loc[s, ['w50', 'w_low', 'w_high']]
    except KeyError:
        warnings.warn(f"站点 {s} 缺少属性数据，跳过")
        continue

    d_bankfull = 0.27 * (w_high / 7.2)**0.6

    # 中值水位 h50（取与 w50 最接近的5条记录回归）
    df_s3['w50_diff'] = np.abs(df_s3['width'] - w50)
    df_s3 = df_s3.sort_values('w50_diff')
    xdata = df_s3.iloc[:5]['width'].values
    ydata = df_s3.iloc[:5]['wse'].values

    if len(np.unique(xdata)) < 2:
        warnings.warn(f"站点 {s} 的 width 值过于相似，跳过")
        continue

    res = linregress(xdata, ydata)
    h50 = res[0] * w50 + res[1] if res[0] >= 0 else ydata.mean()

    # bankfull 面积估算（a50）
    df_s4 = df_s3[(df_s3['width'] >= w_low) & (df_s3['width'] <= w_high)]
    if len(df_s4) < 3:
        continue

    swot_wsemax = df_s4.sort_values('wse', ascending=False).iloc[0]
    d_wsemax = 0.27 * (swot_wsemax['width'] / 7.2)**0.6
    a50 = (q50 * 0.035 / slp**0.5 * w50**(2/3))**(3/5)

    for r_low in R_list:
        for gap in GAP_list:
            for weight in W_list:
                # 拟合下层通道：a_low, h0
                a_low = a50 * (r_low + 1) / r_low / w50**(r_low + 1)
                h0 = h50 - a_low * w50**r_low
                h_low = h0 + a_low * w_low**r_low
                h_high = swot_wsemax['wse'] + (d_bankfull - d_wsemax) + gap * d_bankfull

                # 准备拟合数据
                x_fit = df_s4['width'].values
                y_fit = df_s4['wse'].values
                a_default = (h_high - h0) / w_high**2

                xdata = np.insert(x_fit, 0, [w_low, w_high])
                ydata = np.insert(y_fit, 0, [h_low, h_high])

                try:
                    ls = least_squares(PowerFunc, x0=[h0, a_default, 2], loss=Loss, args=(xdata, ydata))
                    if ls.status == 2:
                        print(f"Optimization status: {ls.status}")
                except Exception as e:
                    print(f"拟合异常于站点 {s}: {e}")
                    continue

                df_res.loc[(s, r_low, gap, weight), ['wse0', 'a', 'b']] = ls.x
                df_res.loc[(s, r_low, gap, weight), ['a50', 'w50', 'q50']] = [a50, w50, q50]
                df_res.loc[(s, r_low, gap, weight), ['w_low', 'w_high', 'h_low', 'h_high', 'slp']] = [w_low, w_high, h_low, h_high, slp]

# 丢弃拟合失败记录
df_res = df_res.dropna()
print("\n拟合完成，共保留结果：", len(df_res))

df_res.to_csv('3/fit_proba_modified.csv')
print('done')

... processing 1/20 ...Optimization status: 2
Optimization status: 2
Optimization status: 2
Optimization status: 2
Optimization status: 2
Optimization status: 2
Optimization status: 2
Optimization status: 2
Optimization status: 2
Optimization status: 2
Optimization status: 2
Optimization status: 2
Optimization status: 2
Optimization status: 2
Optimization status: 2
Optimization status: 2
Optimization status: 2
Optimization status: 2
Optimization status: 2
Optimization status: 2
Optimization status: 2
Optimization status: 2
Optimization status: 2
Optimization status: 2
Optimization status: 2
Optimization status: 2
Optimization status: 2
Optimization status: 2
... processing 3/20 ...Optimization status: 2
Optimization status: 2
Optimization status: 2
Optimization status: 2
Optimization status: 2
Optimization status: 2
Optimization status: 2
Optimization status: 2
Optimization status: 2
Optimization status: 2
Optimization status: 2
Optimization status: 2
Optimization status: 2
Optimizatio

In [2]:
print(df_res['COMID'].nunique())

17
